In [0]:
there 8 types scd

0-6

#scd type 0 - Fixed Dimesnion


In [0]:
%sql
drop table if exists scd_type0;

In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *

data = [(101,'himanshu',25,'M'),
        (201,'sachin',23,'M'),
        (301,'sanant',24,'M'),
        (401,'boon',24,'M'),
        (501,'sudip',28,'M')]

schema = StructType([StructField('id',IntegerType(),True),
                     StructField('name',StringType(),True),
                     StructField('age',IntegerType(),True),
                     StructField('gender',StringType(),True)])

df_type0 = spark.createDataFrame(data,schema)
df_type0.display()

In [0]:
#target table
from delta.tables import DeltaTable

df_type0.write.mode('overwrite').saveAsTable('workspace.default.scd_type0')

df_target = DeltaTable.forName(spark,'scd_type0')
df_target.toDF().display()

In [0]:
#day 1 - batch file - 

data = [(102,'raja',30,'M'),
        (103,'pinky',23,'F'),
        (106,'Shyam',24,'M'),
        (501,'sudip',28,'F')]

schema = StructType([StructField('id',IntegerType(),True),
                     StructField('name',StringType(),True),
                     StructField('age',IntegerType(),True),
                     StructField('gender',StringType(),True)])

df_source = spark.createDataFrame(data,schema)
df_source.display()

In [0]:
%sql

MERGE INTO df_target T
USING df_source S
ON T.id=S.id
WHEN NOT MATCHED THEN
INSERT INTO VALUES(S.id,S.name,S.age,S.gender)

In [0]:
df_target.alias('t').merge(df_source.alias('s'),'t.id=s.id').whenNotMatchedInsertAll().execute()

In [0]:
df_target.toDF().display()

#SCD type 1

In [0]:
df_target.toDF().display()

In [0]:
#2nd batch file

df_source1 = spark.createDataFrame([(101,'Himanshu',27,'M'),
                                    (201,'sachin',25,'M'),
                                    (210,'Ram',24,'M')],schema=(StructType([StructField('id',IntegerType(),True),
                     StructField('name',StringType(),True),
                     StructField('age',IntegerType(),True),
                     StructField('gender',StringType(),True)])))
df_source1.display()

In [0]:
df_target.alias('t').merge(df_source1.alias('s'),'t.id=s.id').whenMatchedUpdate(
    set={'name':'lower(s.name)','age':'s.age'}
).whenNotMatchedInsertAll().execute()
df_target.toDF().display()

# SCD type 2

In [0]:
#curr_rec_ind,active_flag_ind,status
df = spark.sql("""select id,name,age,gender,'Y' as curr_rec_ind from scd_type0""")

#lets re write to the delta table
df.write.format('delta').mode('overwrite').option('overwriteSchema',True).saveAsTable('scd_type0')
DeltaTable.forName(spark,'scd_type0').toDF().display()

In [0]:
#3rd batch file 
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_source2 = spark.createDataFrame([(101,'himanshu',29,'M'),
                                    (201,'sachin',24,'M'),
                                    (210,'Ram',27,'M'),
                                    (215,'sipu',21,'M')],schema=(StructType([StructField('id',IntegerType(),True),
                     StructField('name',StringType(),True),
                     StructField('age',IntegerType(),True),
                     StructField('gender',StringType(),True)])))
df_source2.display()

In [0]:
from delta.tables import DeltaTable

df_target1 = DeltaTable.forName(spark,'scd_type0')
df_target1.toDF().display()
 

In [0]:
df_target1.alias('t').merge(df_source2.alias('s'),'t.id=s.id').whenMatchedUpdate(set={'curr_rec_ind':lit('N')}).whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y')}).execute()

In [0]:
df_target1.toDF().display()

In [0]:

301	sanant	24	M	Y
401	boon	24	M	Y
501	sudip	28	M	Y
102	raja	30	M	Y
103	pinky	23	F	Y
106	Shyam	24	M	Y
101	himanshu	27	M	Y
210	Ram	24	M	Y


id	name	age	gender
101	himanshu	29	M
201	sachin	24	M
210	Ram	27	M
215	sipu	21	M

In [0]:
df_target1.alias('t').merge(df_source2.alias('s'),'t.id=s.id and t.curr_rec_ind="Y"').whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y')}).execute()

In [0]:
df_target1.toDF().display()

In [0]:
df_source3 = spark.createDataFrame(
    [(101, "himanshu", 32, "M")],
    schema=(
        StructType(
            [
                StructField("id", IntegerType(), True),
                StructField("name", StringType(), True),
                StructField("age", IntegerType(), True),
                StructField("gender", StringType(), True),
            ]
        )
    ),
)
df_source3.display()

In [0]:
df_target1.alias('t').merge(df_source3.alias('s'),'t.id=s.id').whenMatchedUpdate(set={'curr_rec_ind':lit('N')}).whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y')}).execute()

In [0]:
df_target1.alias('t').merge(df_source3.alias('s'),'t.id=s.id and t.curr_rec_ind="Y"').whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y')}).execute()

In [0]:
df_target1.toDF().display()

In [0]:
df_target.toDF().display()

In [0]:
#start_ts  end_ts

df=spark.sql("""select id,name,age,gender,'Y' as curr_rec_ind,current_date as start_ts,'null' as end_ts from scd_type0""")

df.write.mode("overwrite").option('mergeSchema','true').saveAsTable("scd_type0")

df_target = DeltaTable.forName(spark,"scd_type0")
df_target.toDF().display()


In [0]:
%sql
update scd_type0 set end_ts=null;
    
select * from scd_type0;

In [0]:
#batch file
df_source4 = spark.createDataFrame(
    [(101, "himanshu", 32, "M"),
     (401,'boon',28,'M'),
     (210,'Ram',27,'M'),
     (215,'sipu',21,'M')],
    schema=(
        StructType(
            [
                StructField("id", IntegerType(), True),
                StructField("name", StringType(), True),
                StructField("age", IntegerType(), True),
                StructField("gender", StringType(), True),
             ]
        )
    ),
)
df_source4.display()

In [0]:
df_target.alias('t').merge(df_source4.alias('s'),'t.id=s.id').whenMatchedUpdate(set={'curr_rec_ind':lit('N'),'end_ts':current_date()}).whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y'),'start_ts':current_date(),'end_ts':'null'}).execute()
df_target.toDF().display()


In [0]:
df_target.alias('t').merge(df_source4.alias('s'),'t.id=s.id and t.curr_rec_ind="Y"').whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y'),'start_ts':current_date(),'end_ts':'null'}).execute()
df_target.toDF().display()

In [0]:
#3rd scenario:-
#batch file
df_source5 = spark.createDataFrame(
    [(101, "himanshu", 32, "M"),
     (515,'rammmm',37,'F')],
    schema=(
        StructType(
            [
                StructField("id", IntegerType(), True),
                StructField("name", StringType(), True),
                StructField("age", IntegerType(), True),
                StructField("gender", StringType(), True),
            ]
        )
    ),
)
df_source5.display()

In [0]:
df_target.alias('t').merge(df_source5.alias('s'),'t.id=s.id').whenMatchedUpdate(set={'curr_rec_ind':lit('N'),'end_ts':current_date()}).whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y'),'start_ts':current_date(),'end_ts':'null'}).execute()
df_target.toDF().display()

In [0]:
df_target.alias('t').merge(df_source5.alias('s'),'t.id=s.id and t.curr_rec_ind="Y"').whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y'),'start_ts':current_date(),'end_ts':'null'}).execute()
df_target.toDF().display()

In [0]:
df=df_target.toDF().withColumn('hash_key',md5(concat_ws('-',col('id'),col('name'),col('age'),col('gender'))))

df.write.format('delta').mode('overwrite').option('mergeSchema','true').saveAsTable('scd_type0')

df_target = DeltaTable.forName(spark,'scd_type0')
df_target.toDF().display()

In [0]:
df_source_hash=df_source5.withColumn('hash_key',md5(concat_ws('-',col('id'),col('name'),col('age'),col('gender'))))

df_source_hash.display()

In [0]:
df_target.alias('t').merge(df_source_hash.alias('s'),'t.id=s.id and t.hash_key!=s.hash_key').whenMatchedUpdate(condition='t.hash_key!=s.hash_key',set={'curr_rec_ind':lit('N'),'end_ts':current_date()}).whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y'),'start_ts':current_date(),'end_ts':'null'}).execute()
df_target.toDF().display()
df_target.alias('t').merge(df_source_hash.alias('s'),'t.id=s.id and t.curr_rec_ind="Y"').whenNotMatchedInsert(values={'id':'s.id','name':'s.name','age':'s.age','gender':'s.gender','curr_rec_ind':lit('Y'),'start_ts':current_date(),'end_ts':'null'}).execute()
df_target.toDF().display()

# SCD type 3

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

spark = SparkSession.builder.appName("SCD3").getOrCreate()


In [0]:
# Target data
target_data = [
    (1, "John", "New York", None),
    (2, "Alice", "Boston", None),
    (5,"smith","NY",None),
    (6,"ram","LA",None)
]

target_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("current_location", StringType(), True),
    StructField("previous_location", StringType(), True)
])

target_df = spark.createDataFrame(target_data, schema=target_schema) 

target_df.write.format('delta').mode('overwrite').option('mergeSchema','true').saveAsTable('scd_type3')
target_df = DeltaTable.forName(spark,'scd_type3')
target_df.toDF().display()


In [0]:
# after some dya i got a source file

# Source data
source_data = [
    (1, "John", "Chicago"),
    (2, "Alice", "Boston"),
    (3, "Bob", "Seattle")
]

source_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True)
])

source_df = spark.createDataFrame(source_data, schema=source_schema) 
source_df.display()


In [0]:
target_df.alias('t').merge(
    source_df.alias('s'),
    't.id = s.id'
).whenMatchedUpdate(
    set={
        'previous_location': 't.current_location',
        'current_location': 's.location'
    }
).whenNotMatchedInsert(
    values={
        'id': 's.id',
        'name': 's.name',
        'current_location': 's.location',
        'previous_location': 'null'
    }
).execute()

In [0]:
target_df.toDF().display()
